This code is mainly by Edward Raff - https://github.com/EdwardRaff/Inside-Deep-Learning

Modified by Arunesh Sinha

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd

<ipython-input-1-93e2626d8a4a>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

<ipython-input-2-475911aa0aa9>:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


In [3]:
import torch
print(torch.__version__)

2.5.1+cu121


In [4]:
!python --version

Python 3.11.11


In [5]:
torch_scalar = torch.tensor(3.14)
torch_vector = torch.tensor([1, 2, 3, 4])
torch_matrix = torch.tensor([[1, 2,],
                             [3, 4,],
                             [5, 6,],
                             [7, 8,]])
#You don't have to format it like I did, thats just for clarity
torch_tensor3d = torch.tensor([
                            [
                            [ 1,  2,  3],
                            [ 4,  5,  6],
                            ],
                            [
                            [ 7,  8,  9],
                            [10, 11, 12],
                            ],
                            [
                            [13, 14, 15],
                            [16, 17, 18],
                            ],
                            [
                            [19, 20, 21],
                            [22, 23, 24],
                            ]
                              ])

In [6]:
print(torch_scalar.shape)
print(torch_vector.shape)
print(torch_matrix.shape)
print(torch_tensor3d.shape)

torch.Size([])
torch.Size([4])
torch.Size([4, 2])
torch.Size([4, 2, 3])


In [7]:
x_np = np.random.random((4,4))
print(x_np)

[[0.6871098  0.50773132 0.17269064 0.45411499]
 [0.5355536  0.0304786  0.0155566  0.45035733]
 [0.41691169 0.1996019  0.67393001 0.93133536]
 [0.16359732 0.14186881 0.24691366 0.21292915]]


In [8]:
x_pt = torch.tensor(x_np)
print(x_pt)

tensor([[0.6871, 0.5077, 0.1727, 0.4541],
        [0.5356, 0.0305, 0.0156, 0.4504],
        [0.4169, 0.1996, 0.6739, 0.9313],
        [0.1636, 0.1419, 0.2469, 0.2129]], dtype=torch.float64)


In [9]:
print(x_np.dtype, x_pt.dtype)

float64 torch.float64


In [10]:
#Lets force them to be 32-bit floats
x_np = np.asarray(x_np, dtype=np.float32)
x_pt = torch.tensor(x_np, dtype=torch.float32)
print(x_np.dtype, x_pt.dtype)

float32 torch.float32


In [12]:
b_np = (x_np > 0.5)
print(b_np)
print(b_np.dtype)

[[ True  True False False]
 [ True False False False]
 [False False  True  True]
 [False False False False]]
bool


In [13]:
b_pt = (x_pt > 0.5)
print(b_pt)
print(b_pt.dtype)

tensor([[ True,  True, False, False],
        [ True, False, False, False],
        [False, False,  True,  True],
        [False, False, False, False]])
torch.bool


In [14]:
np.sum(x_np)

5.8406806

In [15]:
torch.sum(x_pt)

tensor(5.8407)

In [16]:
np.transpose(x_np)

array([[0.6871098 , 0.5355536 , 0.4169117 , 0.16359732],
       [0.5077313 , 0.0304786 , 0.1996019 , 0.14186881],
       [0.17269063, 0.0155566 , 0.67393   , 0.24691366],
       [0.45411497, 0.45035735, 0.93133533, 0.21292916]], dtype=float32)

In [17]:
# torch transpose not same as numpy transpose, remember always use https://pytorch.org/docs/stable/index.html
torch.transpose(x_pt, 0, 1)

tensor([[0.6871, 0.5356, 0.4169, 0.1636],
        [0.5077, 0.0305, 0.1996, 0.1419],
        [0.1727, 0.0156, 0.6739, 0.2469],
        [0.4541, 0.4504, 0.9313, 0.2129]])

In [18]:
print(torch.transpose(torch_tensor3d, 0, 2).shape)

torch.Size([3, 2, 4])


In [19]:
import timeit
x = torch.rand(2**11, 2**11)
time_cpu = timeit.timeit("x@x", globals=globals(), number=100)

In [20]:
print("Is CUDA available? :", torch.cuda.is_available())
device = torch.device("cuda")

Is CUDA available? : True


In [21]:
x = x.to(device)
time_gpu = timeit.timeit("x@x", globals=globals(), number=100)

In [22]:
x = torch.rand(128, 128).to(device)
y = torch.rand(128, 128)
x*y

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [23]:
x = x.cpu()
x*y

tensor([[0.1788, 0.1225, 0.1978,  ..., 0.4529, 0.0151, 0.0048],
        [0.0879, 0.1744, 0.2375,  ..., 0.0436, 0.2493, 0.1883],
        [0.4422, 0.0574, 0.4379,  ..., 0.5199, 0.2239, 0.0132],
        ...,
        [0.2681, 0.1797, 0.0340,  ..., 0.2858, 0.0941, 0.1511],
        [0.6474, 0.0864, 0.1872,  ..., 0.5183, 0.3104, 0.3278],
        [0.4671, 0.2021, 0.3420,  ..., 0.3366, 0.0372, 0.1342]])

In [ ]:
# A helper function to move various types of objects to a device
def moveTo(obj, device):
    """
    obj: the python object to move to a device, or to move its contents to a device
    device: the compute device to move objects to
    """
    if isinstance(obj, list):
        return [moveTo(x, device) for x in obj]
    elif isinstance(obj, tuple):
        return tuple(moveTo(list(obj), device))
    elif isinstance(obj, set):
        return set(moveTo(list(obj), device))
    elif isinstance(obj, dict):
        to_ret = dict()
        for key, value in obj.items():
            to_ret[moveTo(key, device)] = moveTo(value, device)
        return to_ret
    elif hasattr(obj, "to"):
        return obj.to(device)
    else:
        return obj

some_tensors = [torch.tensor(1), torch.tensor(2)]
print(some_tensors)
print(moveTo(some_tensors, device))

In [24]:
# _ gives inplace versions of functions (do not use inplace operations, at least when you need gradients)
# inplace operations can help save memory, if used properly
op1 = torch.ones(3,3)
op2 = torch.rand(3,3)
print(op1)
print(op2)
print(op1.add(op2))
print(op1)
print(op1.add_(op2)) # inplace operation, not recommended

print(op1)
print(op1 + 1)
print(op1)
op1 += 1 # inplace operation, not recommended
print(op1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.2515, 0.3062, 0.3974],
        [0.8909, 0.6970, 0.7342],
        [0.4137, 0.3289, 0.6258]])
tensor([[1.2515, 1.3062, 1.3974],
        [1.8909, 1.6970, 1.7342],
        [1.4137, 1.3289, 1.6258]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1.2515, 1.3062, 1.3974],
        [1.8909, 1.6970, 1.7342],
        [1.4137, 1.3289, 1.6258]])
tensor([[1.2515, 1.3062, 1.3974],
        [1.8909, 1.6970, 1.7342],
        [1.4137, 1.3289, 1.6258]])
tensor([[2.2515, 2.3062, 2.3974],
        [2.8909, 2.6970, 2.7342],
        [2.4137, 2.3289, 2.6258]])
tensor([[1.2515, 1.3062, 1.3974],
        [1.8909, 1.6970, 1.7342],
        [1.4137, 1.3289, 1.6258]])
tensor([[2.2515, 2.3062, 2.3974],
        [2.8909, 2.6970, 2.7342],
        [2.4137, 2.3289, 2.6258]])


In [25]:
x = torch.rand(1, requires_grad=True)
print(x)

tensor([0.0498], requires_grad=True)


In [26]:
y = x + 2
print(y)
z = y*y
#z = y.mean()

#y.retain_grad()
z.backward()
print(x.grad) # dz/dx
#print(y.grad)
#x.gra

# No grad - 3 known ways
#x.requires_grad_(False)
#x.detach()
with torch.no_grad():
  y = x*x
  #print(y)

tensor([2.0498], grad_fn=<AddBackward0>)
tensor([4.0995])


In [ ]:
x = torch.rand(2, requires_grad=True)
print(x)

tensor([0.6923, 0.2652], requires_grad=True)


In [ ]:
y = x + 2
print(y)
#z = y*y
z = y.mean()

y.retain_grad()
z.backward()
print(x.grad) # dz/dx
print(y.grad)

# No grad - 3 known ways
#x.requires_grad_(False)
#x.detach()
with torch.no_grad():
  y = x*x
  #print(y)

tensor([2.6923, 2.2652], grad_fn=<AddBackward0>)
tensor([0.5000, 0.5000])
tensor([0.5000, 0.5000])


In [27]:
!python --version

Python 3.11.11


In [ ]:
#@title
def f(x):
    return torch.pow((x-2.0), 2)

x_axis_vals = np.linspace(-7,9,100)
y_axis_vals = f(torch.tensor(x_axis_vals)).numpy()

sns.lineplot(x=x_axis_vals, y=y_axis_vals, label='$f(x)=(x-2)^2$')
#@title
def fP(x): #Defining the derivative of f(x) manually
    return 2*x-4

y_axis_vals_p = fP(torch.tensor(x_axis_vals)).numpy()

#First, lets draw a black line at 0, so that we can easily tell if something is positive or negative
sns.lineplot(x=x_axis_vals, y=[0.0]*len(x_axis_vals), label="0", color='black')
sns.lineplot(x=x_axis_vals, y=y_axis_vals, label='$f(x) = (x-2)^2$')
sns.lineplot(x=x_axis_vals, y=y_axis_vals_p, label="$f'(x)=2 x - 4$")
#@title
x = torch.tensor([-3.5], requires_grad=True)
print(x.grad)
#@title
value = f(x)
print(value)
#@title
value.backward()
print(x.grad)
#@title
x = torch.tensor([-3.5], requires_grad=True)

x_cur = x.clone()
x_prev = x_cur*100 #Make the initial "previous" solution larger
epsilon = 1e-5
eta = 0.1

while torch.linalg.norm(x_cur-x_prev) > epsilon:
    x_prev = x_cur.clone() #We need to make a clone here so that x_prev and x_cur don't point to the same object

    #Compute our function, gradient, and update
    value = f(x)
    value.backward()
    x.data -= eta * x.grad
    x.grad.zero_() #We need to zero out the old gradient, as py-torch will not do that for us

    #What are we currently now?
    x_cur = x.data

print(x_cur)
#@title
x_param = torch.nn.Parameter(torch.tensor([-3.5]), requires_grad=True)
#@title
optimizer = torch.optim.SGD([x_param], lr=eta)
#@title
for epoch in range(60):
    optimizer.zero_grad() #x.grad.zero_()
    loss_incurred  = f(x_param)
    loss_incurred.backward()
    optimizer.step() #x.data -= eta * x.grad
print(x_param.data)
#@title
from torch.utils.data import Dataset
from sklearn.datasets import fetch_openml

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
print(X.shape)
#@title
class SimpleDataset(Dataset):

    def __init__(self, X, y):
        super(SimpleDataset, self).__init__()
        self.X = X
        self.y = y

    def __getitem__(self, index):
        #This "work" could have gone in the constructor, but you should get into
        inputs = torch.tensor(self.X[index,:], dtype=torch.float32)
        targets = torch.tensor(int(self.y[index]), dtype=torch.int64)
        return inputs, targets

    def __len__(self):
        return self.X.shape[0]
#Now we can make a PyTorch dataset
dataset = SimpleDataset(X, y)
#@title
print("Length: ", len(dataset))
example, label = dataset[0]
print("Features: ", example.shape) #Will return 784
print("Label of index 0: ", label)
#@title
plt.imshow(example.reshape((28,28)))
#@title
train_size = int(len(dataset)*0.8)
test_size = len(dataset)-train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_size, test_size))
print("{} examples for training and {} for testing".format(len(train_dataset), len(test_dataset)))

NameError: ignored